### Построение модели для задачи классификации

Некоторым оператором сотовой связи было установлено, что многие клиенты пользуются архивными тарифами. Оператор хочет предложить им на выбор два тарифа: Смарт и Ультра.

Целью проекта является построение модели для задачи классификации, которая выберет подходящий тариф.

### 1. Изучение данных 

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


столбец `is_ultra` показывает, каким тарифом пользовался клиент: 1 - Ультра, 0 - Смарт.

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
data.shape

(3214, 5)

Изучив данные видно, что пустых значений нет, все числа имеют подходящий тип. Данные полностью готовы к посторению по ним модели.

### 2. Разделение данных на датасеты

Разделим имеющиеся данные на три части: обучающую, валидационную и тестовую. По обучающей части будем учить модель, по валидационной - выбирать лучшую, а по тестовой -  тестировать финальное качество выбранной модели.

Будем делить имеющийся датасет в следующих пропорциях: обучающая часть - 60%, валидационная - 20% и тестовая - 20%.

In [6]:
# импортируем все библиотеки, которые понадобятся нам в дальнейшем
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

Произведем разбиение датасета

In [7]:
data_train, data_valid = train_test_split(data, test_size=0.4, random_state=12345)

In [8]:
print(data_train.shape)
print(data_valid.shape)

(1928, 5)
(1286, 5)


Теперь разделим `data_valid` пополам. Одна из этих частей будет тестовой, другая валидационной.

In [9]:
data_validation, data_test = train_test_split(data_valid, test_size=0.5, random_state=12345)

In [10]:
print(data_train.shape) # обучающая выборка
print(data_validation.shape) # валидационная выборка
print(data_test.shape) # тестовая выборка

(1928, 5)
(643, 5)
(643, 5)


В этом раздела мы получили три набора данных (обучающая выборка, валидационная выборка и тестовая выборка), на основании которых в дальнейшем будем строить обучение моделей и проверять их качество.

### 3. Исследование качества разных моделей

Выделим из этих трех датасетов признаки и целевой признак.

In [11]:
features_train = data_train.drop('is_ultra', axis=1)
target_train = data_train['is_ultra']

features_validation = data_validation.drop('is_ultra', axis=1)
target_validation = data_validation['is_ultra']

features_test = data_test.drop('is_ultra', axis=1)
target_test = data_test['is_ultra']

Сравним качество работы трех моделей: дерева решений, случайного леса и логистической регрессии.

*1) Дерево решений*

Методом перебора выявим, какому гиперпараметру `max_depth` (максимальная глубина дерева) соответствует максимальная правильность (accuracy).

In [12]:
for depth in range(2, 20):
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions = model_tree.predict(features_validation)
    accuracy = accuracy_score(target_validation, predictions)
    print(f'max_depth = {depth}, accuracy = {accuracy}')

max_depth = 2, accuracy = 0.7822706065318819
max_depth = 3, accuracy = 0.7853810264385692
max_depth = 4, accuracy = 0.7791601866251944
max_depth = 5, accuracy = 0.7791601866251944
max_depth = 6, accuracy = 0.7838258164852255
max_depth = 7, accuracy = 0.7822706065318819
max_depth = 8, accuracy = 0.7791601866251944
max_depth = 9, accuracy = 0.7822706065318819
max_depth = 10, accuracy = 0.7744945567651633
max_depth = 11, accuracy = 0.7620528771384136
max_depth = 12, accuracy = 0.7620528771384136
max_depth = 13, accuracy = 0.7558320373250389
max_depth = 14, accuracy = 0.7589424572317263
max_depth = 15, accuracy = 0.7465007776049767
max_depth = 16, accuracy = 0.7340590979782271
max_depth = 17, accuracy = 0.7356143079315708
max_depth = 18, accuracy = 0.7309486780715396
max_depth = 19, accuracy = 0.7278382581648523


Максимальное значение параметра `accuracy` ( 0.785) достигается при значении `max_deph = 3`

*2) Случайный лес*

Методом перебора выявим, какому гиперпараметру `n_estimators` (количество деревьев) соответствует максимальная правильность (`accuracy`).

In [13]:
for estim in range(2, 40, 2):
    model_forest = RandomForestClassifier(random_state=12345, n_estimators=estim)
    model_forest.fit(features_train, target_train)
    predictions = model_forest.predict(features_validation)
    accuracy = accuracy_score(target_validation, predictions)
    print(f'n_estimators = {estim}, accuracy = {accuracy}')

n_estimators = 2, accuracy = 0.7636080870917574
n_estimators = 4, accuracy = 0.7713841368584758
n_estimators = 6, accuracy = 0.7807153965785381
n_estimators = 8, accuracy = 0.7822706065318819
n_estimators = 10, accuracy = 0.7853810264385692
n_estimators = 12, accuracy = 0.7869362363919129
n_estimators = 14, accuracy = 0.7838258164852255
n_estimators = 16, accuracy = 0.7869362363919129
n_estimators = 18, accuracy = 0.7931570762052877
n_estimators = 20, accuracy = 0.7869362363919129
n_estimators = 22, accuracy = 0.7884914463452566
n_estimators = 24, accuracy = 0.7900466562986003
n_estimators = 26, accuracy = 0.7853810264385692
n_estimators = 28, accuracy = 0.7838258164852255
n_estimators = 30, accuracy = 0.7838258164852255
n_estimators = 32, accuracy = 0.7822706065318819
n_estimators = 34, accuracy = 0.7807153965785381
n_estimators = 36, accuracy = 0.7807153965785381
n_estimators = 38, accuracy = 0.7838258164852255


Максимальное `accuracy` (0.793) получается при `n_estimators = 18`.

*3) Логистическая регрессия*

In [81]:
model_lr = LogisticRegression(random_state=12345)
model_lr.fit(features_train, target_train)
predictions = model_lr.predict(features_validation)
accuracy = accuracy_score(target_validation, predictions)
accuracy

D:\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7542768273716952

В методе логистической регрессии значение `accuracy` составило 0.754.

Сравнив качество работы трех алгоритмов классификации (дерево решений, случайный лес и логистическая регрессия), было обнаружено, что наилучшим образом себя показывает случайный лес (`accuracy = 0.793`), а наихудшим - логистическая регрессия (`accuracy = 0.754`).

### 4. Проверка качества модели

Проверим качество модели, выбранной нами как наилучшей в предыдущем разделе (случайный лес), на тестовой выборке.

In [82]:
predictions = model_forest.predict(features_test) # проверка модели на тестовых данных путем получения предсказанных значений
accuracy = accuracy_score(target_test, predictions) 
print('accuracy =', accuracy)

accuracy = 0.7947122861586314


Такое значение `accuracy` считаем приемлемым в рамках данной работы. Модель не переобучена и показывает неплохие результаты на тестовой выборке.

### 5. Оценка адекватности модели

Оценку адекватности проведем следующим образом. Предположим, что в тестовой выборке некая модель предсказала все параметры равными 1 (или 0). Посчитаем параметр `accuracy` в таком случае и сравним его с нашей моделью.

In [83]:
# создадим список, состоящий только из единиц
predictions_sanity = [1 for i in range(len(features_test))]

In [84]:
accuracy_sanity_check = accuracy_score(target_test, predictions_sanity)
accuracy_sanity_check

0.3157076205287714

In [85]:
# создадим список, состоящий только из нулей
predictions_sanity = [0 for i in range(len(features_test))]

In [86]:
accuracy_sanity_check = accuracy_score(target_test, predictions_sanity)
accuracy_sanity_check

0.6842923794712286

Можно видеть, что значение `accuracy` в обоих случаях меньше, чем то, которое мы получили с ипользованием обученной модели случайного леса. Считаем, что наша модель является адекватной.

### 6. Вывод

В работе были обучены и протестированы 3 модели задачи классификации: дерево решений, случайный лес и логистическая регрессия. Каждая из моделей обучалась на обучающей выборке данных и тестировалась на валидационной. По результатам тестирования моделей на валидационной выборке была выбрана наилучшая модель - случайный лес, - показавшая значине `accuracy = 0.793`. Окончательно протестировав эту модель на тестовой выборке, было получено значение `accuracy = 0.794`. Оценка адекватности модели показала, что ее можно использовать для задачи выбора подходящего тарифа для пользователя. Таким образом, поставленные вначале задачи были полностью решены.